## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-13-02-23-33 +0000,nyt,Russia Sues Holder of Frozen Assets Europe Wan...,https://www.nytimes.com/2025/12/12/world/europ...
1,2025-12-13-02-22-44 +0000,nyt,U.S. Boarded Ship and Seized Cargo Heading to ...,https://www.nytimes.com/2025/12/12/us/politics...
2,2025-12-13-02-19-23 +0000,nyt,Venezuela Oil Tanker Seized by U.S. Was Part o...,https://www.nytimes.com/2025/12/12/world/ameri...
3,2025-12-13-02-16-51 +0000,bbc,"New photos from Epstein estate show Trump, And...",https://www.bbc.com/news/articles/crk777lj763o...
4,2025-12-13-02-16-48 +0000,nyt,"In Florida’s Panhandle, a Hearing on School Va...",https://www.nytimes.com/2025/12/12/us/florida-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2416/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,48
23,new,20
106,year,13
15,venezuela,13
14,china,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
106,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...,126
332,2025-12-12-02-38-00 +0000,wsj,President Trump signs an executive order that ...,https://www.wsj.com/politics/policy/trump-sign...,119
146,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...,104
206,2025-12-12-15-51-53 +0000,nypost,"Woody Allen, Andrew Mountbatten-Windsor, Bill ...",https://nypost.com/2025/12/12/us-news/woody-al...,104
3,2025-12-13-02-16-51 +0000,bbc,"New photos from Epstein estate show Trump, And...",https://www.bbc.com/news/articles/crk777lj763o...,103


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
106,126,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...
146,56,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...
296,50,2025-12-12-09-08-17 +0000,nyt,Venezuela’s Nobel Winner Will Go to Norway for...,https://www.nytimes.com/2025/12/06/world/ameri...
250,37,2025-12-12-12-18-16 +0000,cbc,Indiana Republicans help defy Trump's push to ...,https://www.cbc.ca/news/world/us-indiana-redis...
113,36,2025-12-12-21-03-00 +0000,wsj,Reddit Sues Australian Government for Under-16...,https://www.wsj.com/business/media/reddit-sues...
138,35,2025-12-12-20-01-34 +0000,latimes,Leaders of secretive California religious grou...,https://www.latimes.com/california/story/2025-...
151,34,2025-12-12-19-14-51 +0000,bbc,EU backs indefinite freeze on Russia's frozen ...,https://www.bbc.com/news/articles/c98nnd01g91o...
70,34,2025-12-12-22-49-54 +0000,startribune,"Before she takes power, St. Paul Mayor-elect K...",https://www.startribune.com/st-paul-mn-mayor-e...
179,32,2025-12-12-17-34-00 +0000,wsj,Amtrak managers are giving up $16 million of t...,https://www.wsj.com/politics/policy/amtrak-wor...
217,32,2025-12-12-15-01-01 +0000,nypost,"Stefanik, Jordan demand Hochul end NY funding ...",https://nypost.com/2025/12/12/us-news/stefanik...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
